In [1]:
import datapane as dp
import pandas as pd
from pytrends.request import TrendReq
import altair as alt

pytrends = TrendReq(hl='en-US', tz=360)

queries = ['data visualization', 'tableau', 'plot python']
results = {}


In [2]:
pytrends.build_payload(queries, cat=0, timeframe='today 5-y', geo='US', gprop='')


In [5]:
r_o_t = pytrends.interest_over_time().reset_index()
from scipy.stats import zscore
for query in queries:
    r_o_t[query] = zscore(r_o_t[query])

In [6]:
regions = pytrends.interest_by_region(resolution='REGION', inc_geo_code=True)

In [7]:
df = r_o_t.drop(columns=['isPartial']).melt('date', var_name='keyword')

In [8]:
from scipy.stats import zscore

In [9]:
base = alt.Chart(df).encode(x='date', y='value', color='keyword').mark_line(opacity=0.2)
base + base.transform_loess('date', 'value', groupby=['keyword']).mark_line().interactive()

alt.LayerChart(...)

alt.Chart(...)

In [10]:
regions

,geoCode,data visualization,tableau,plot python
geoName,,,,
Alabama,US-AL,7,81,12
Alaska,US-AK,0,100,0
Arizona,US-AZ,5,88,7
Arkansas,US-AR,5,95,0
California,US-CA,6,83,11
Colorado,US-CO,6,84,10
Connecticut,US-CT,5,88,7
Delaware,US-DE,0,100,0
District of Columbia,US-DC,7,89,4


In [11]:
import altair as alt
from vega_datasets import data

def gen_chart(query):
    geo = regions.reset_index()[['geoName', query]]
    states = alt.topo_feature(data.us_10m.url, 'states')
    state_fips = pd.read_csv('https://raw.githubusercontent.com/kjhealy/fips-codes/master/state_fips_master.csv')
    geo_query = geo.merge(state_fips, right_on='state_name', left_on='geoName',  how = 'outer')

    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()

    geo_query['scaled'] = scaler.fit_transform(geo_query[query].values.reshape(-1,1)).round(1)
    # US states background
    base_map = alt.Chart(states).mark_geoshape(
        stroke='black',
        fill='#eee',
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(geo_query, 'fips', ['scaled', 'state_name'])
    ).encode(
        tooltip='state_name:N'
    )

    encoded = base_map.encode(
        fill=alt.Color('scaled:Q', scale=alt.Scale(scheme='lightmulti'))
    ).properties(
        title='Breakdown by state',
        width=650,
        height=400
    ).project('albersUsa')

    map_plot = base_map + encoded 
    return map_plot

map_plot = gen_chart('tableau') | gen_chart('plot python') 

In [13]:
from datapane import LocalReport, Plot, Table, Markdown, Asset

r = [
    Markdown(f"""# Google Trend Analysis 
What are the Google trends? Which US states are most interested? This script pulls data from the Google Trends API to find out...
"""),
    Table.create(regions),
    Markdown(f"""
# Keyword geography interest

We can also query Google Trends to find which states search the most, and plot it on a map."""),
Plot.create(map_plot)
]

LocalReport.create(*r, path='index.html').open()